# Загрузка базы вопросов ЧГК

Нужно загрузить все доступные вопросы и ответы к ним с базы http:///db.chgk.info . Результаты необходимо сохранить в каком-либо формате в форме вопрос-ответ (pickle, sqlite, текстовый файл и т.д.). 

In [2]:
import lxml.html
import lxml.etree
import requests

In [1]:
def parse_page(url):
    response = requests.get(url)
    if (response.status_code == 200):
        links = []
        tree = lxml.html.fromstring(response.text)
        even = tree.xpath('//tr[contains(@class, "even")]//a[contains(@rel, "nofollow")]/@href')
        odds = tree.xpath('//tr[contains(@class, "odd")]//a[contains(@rel, "nofollow")]/@href')
        for element in even:
            links.append('https://db.chgk.info' + element)
        for element in odds:
            links.append('https://db.chgk.info' + element)
        return links

In [2]:
def parse_event(event_page_link):
    num_of_errors = 0
    response = requests.get(event_page_link)
    if (response.status_code == 200):
        res = []
        tree = lxml.html.fromstring(response.text)
        containers = tree.xpath('//div[contains(@class, "question")]')
        for a in containers:
            question = a.getchildren()[0].text_content()
            try:
                answer = a.getchildren()[1].getchildren()[1].text_content()            
            except:
                num_of_errors = num_of_errors + 1
                answer = ''
            tuple_ans = (question, answer)
            res.append(tuple_ans)
    if num_of_errors>0:
        print(f"На странице {event_page_link} было {num_of_errors} ошибок")
    return res

In [3]:
def parse_to_db(db_dir):
    page_num = 0
    link = ''
    conn = sqlite3.connect(db_dir)
    c = conn.cursor()

    c.execute('''CREATE TABLE www
                 (questions, answers)''')

    while link != 'https://db.chgk.info/tour/sanbel00':
        print(f"Обрабатывается {page_num} страница.")
        page_url = 'https://db.chgk.info/last?page=' + str(page_num)
        urls = parse_page(page_url)
        for url in urls:
            results = parse_event(url)
            c.executemany('INSERT INTO www VALUES (?,?)', results)        
        link = urls[-1]
        page_num = page_num + 1
    conn.commit()
    conn.close()